Step 1: Detection of eyes and when the eye is blinking, I deteced the gaze and find the direction in which we are looking - top, bottom, left, right.
    
Step 2: Creation of virtual keyboard on the screen in which we have letters from the keyboard.
    
Step 3: Linking of both the steps and giving commands to the keyboard.  

Useful for people suffering from quadriplegia, who cannot use their limbs. 

# Finding two lines on one eye: horizontal and vertical

In [2]:
import cv2
import numpy as np
import dlib

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

while True:
    
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        left_point = (landmarks.part(36).x, landmarks.part(36).y)
        right_point = (landmarks.part(39).x, landmarks.part(39).y)
        center_top = midpoint(landmarks.part(37), landmarks.part(38))
        center_bottom = midpoint(landmarks.part(41), landmarks.part(40))
        
        hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
        ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# To check whether the eye blinks or not
This can be checked by finding the lengths of both above lines. The length of horizontal line remains the same either on closing or openinng the eye, but the vertical length changes on closing the line.

In [5]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily


cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

while True:
    
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0))
        
        
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# To check the exact gaze of my eye by drawing region around the eye , creating threshold to separate the iris and the pupil part of the eye then removing the extra eye portion using masking.

In [16]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily


cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

while True:
    
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        #Detect blinking
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0))
        
        #Gaze detection - drawing polygon on the left eye and getting the exact region of the eye
        left_eye_region = np.array([(landmarks.part(36).x, landmarks.part(36).y),
                                   (landmarks.part(37).x, landmarks.part(37).y),
                                   (landmarks.part(38).x, landmarks.part(38).y),
                                   (landmarks.part(39).x, landmarks.part(39).y),
                                   (landmarks.part(40).x, landmarks.part(40).y),
                                   (landmarks.part(41).x, landmarks.part(41).y)], np.int32)
        #drawing polygon on the left eye
        #cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
        
        
        height, width, _ = frame.shape    #_ is for no of channels.
        mask = np.zeros((height, width), np.uint8)  #this is a black screen of size same as our frame
        #We will create the eye region in this mask
        cv2.polylines(mask, [left_eye_region], True, (255,255,255), 2)  #creating the polygon of the eye region
        cv2.fillPoly(mask, [left_eye_region], 255)  #filling the polygon
        #mask gives exact eye, so we put this mask over the frame so as to get exact eye and the rest extra portion of eye is removed.
        left_eye = cv2.bitwise_and(gray, gray, mask = mask)
        
        
        #detecting the eye region
        min_x = np.min(left_eye_region[:, 0])  #left most pt is min x
        max_x = np.max(left_eye_region[:, 0])  #rightt most pt is max x
        min_y = np.min(left_eye_region[:, 1])  #top most pt is min y
        max_y = np.max(left_eye_region[:, 1])  #bottomt most pt is max y
        
        gray_eye = left_eye[min_y : max_y, min_x : max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
        
        
        eye = cv2.resize(gray_eye, None, fx = 5, fy = 5)  #increasing the size of window 5 times than the original one.
        threshold_eye = cv2.resize(threshold_eye, None, fx = 5, fy = 5)
        cv2.imshow("Eye", eye)  #showing the eye region
        cv2.imshow("Threshold", threshold_eye)
        #cv2.imshow("Mask", mask)
        cv2.imshow("Left Eye", left_eye)
        
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# To find the gaze from the threshold
To find the direction of the eye : If I am looking on right side, then I have the pupil and the iris pointing towards the right and restof the eye is completely white. Using thresholding I divided one part as while and the rest black part is the iris. Similarly this is true if I look at the left side and If I look at the center then the white part is balanced. 

In [ ]:
So we divide the threshold of eye into 2 parts - left and right and then see where white part is more and we count this white by using opencv funcn ,here white part is-1 and black part is 0.
Since it becomes difficult to look at the screen when I look on either sides : left or right, so I used a different frame which shows some colour change when I turn sideways.

In [23]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily


cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

def get_gaze_ratio(eye_points, landmarks):
    
        #Gaze detection - drawing polygon on the left eye and getting the exact region of the eye
        left_eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                                   (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                                   (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                                   (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                                   (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                                   (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
        #drawing polygon on the left eye
        #cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
        
        
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)  #this is a black screen of size same as our frame
        #We will create the eye region in this mask
        cv2.polylines(mask, [left_eye_region], True, (255,255,255), 2)  #creating the polygon of the eye region
        cv2.fillPoly(mask, [left_eye_region], 255)  #filling the polygon
        #mask gives exact eye, so we put this mask over the frame so as to get exact eye and the rest extra portion of eye is removed.
        eye = cv2.bitwise_and(gray, gray, mask = mask)
        
        
        #detecting the eye region
        min_x = np.min(left_eye_region[:, 0])  #left most pt is min x
        max_x = np.max(left_eye_region[:, 0])  #rightt most pt is max x
        min_y = np.min(left_eye_region[:, 1])  #top most pt is min y
        max_y = np.max(left_eye_region[:, 1])  #bottomt most pt is max y
        
        gray_eye = eye[min_y : max_y, min_x : max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
        
        #dividing the threshold into 2 parts - left and write threshold
        height, width = threshold_eye.shape
        left_side_threshold = threshold_eye[0: height, 0: int(width/2)]
        right_side_threshold = threshold_eye[0: height, int(width/2): width]
        
        #calculating the amount of white part in both the thresholds - left and right
        left_side_white = cv2.countNonZero(left_side_threshold)
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        #handling zero divison error
        if left_side_white == 0:
            gaze_ratio = 1   #means we are looking at the left
        elif right_side_white == 0:
            gaze_ratio = 5   #means we are looking at the right
        else:
            gaze_ratio = left_side_white/ right_side_white
        

        return gaze_ratio
    
    

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

while True:
    
    _, frame = cap.read()
    #showing direction when we turn the face eithre left and right
    new_frame = np.zeros((500, 500, 3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        #Detect blinking
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0))
        
        
        #gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio = (gaze_ratio_left_eye + gaze_ratio_right_eye)/ 2
        
        if gaze_ratio < 0.8: 
            cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (0, 0, 255)
        elif 0.8 < gaze_ratio < 1.2:
            cv2.putText(frame, "CENTER", (50, 100), font, 2, (0,0,255), 3)
        else:
            cv2.putText(frame, "LEFT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (255, 0, 0)
        
        
        
        
        
        
        
    cv2.imshow("Frame", frame)
    cv2.imshow('New Frame', new_frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# Creation of virtual keyboard on the screen
The cursor will be moving upon the letters and as we want to press a letter then I blink my eye.

In [27]:
import cv2
import numpy as np

#creation of black screen
keyboard = np.zeros((1000, 1500, 3), np.uint8)


def letter(x, y, text):  #this function is used to draw bixes of rectangles and the text inside it. 
    #Keys
    width = 200
    height = 200
    thickness = 3
    cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (255, 255, 0), thickness)

    #settings of the text
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_thickness = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_thickness)[0]  #gives size of the letter
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x   #pixels are in integer.
    text_y = int((height + height_text) / 2) + y
    cv2.putText(keyboard, text , (text_x, text_y), font_letter , font_scale, (255, 255, 0), font_thickness)


    
letter(0, 0, "A")
letter(200, 0, "B")
letter(400, 0, "C")
letter(600, 0, "D")


cv2.imshow('keyboard', keyboard)
cv2.waitKey(0)
cv2.destroyAllWindows()

# During the creation of all letters in the keyboard it will take more time time to reach to the lower letters, so I splitted the Keyboard into 2 parts. So it is a faster way to select letters. I create 5 keys in each row and there are 3 such rows and each box has width and height = 200. So frame is of size - height = 600 and length = 1000

In [33]:
import cv2
import numpy as np

#creation of black screen
keyboard = np.zeros((600, 1000, 3), np.uint8)

keys_set1 = {0 : "Q", 1: 'W', 2: 'E', 3: 'R', 4: 'T',
            5: 'A', 6: 'S', 7: 'D', 8: 'F', 9: 'G',
            10: 'Z', 11: 'X', 12: 'C', 13: 'V', 14: 'B'}

def letter(letter_index, text, letter_light):  #this function is used to draw bixes of rectangles and the text inside it. 
    #Keys
    if letter_index == 0:  #0th row
        x = 0
        y = 0
    elif letter_index == 1:
        x = 200
        y = 0
    elif letter_index == 2:
        x = 400
        y = 0
    elif letter_index == 3:
        x = 600
        y = 0  
    elif letter_index == 4:
        x = 800
        y = 0
    elif letter_index == 5:   #from this it is 1st row
        x = 0
        y = 200
    elif letter_index == 6:
        x = 200
        y = 200
    elif letter_index == 7:
        x = 400
        y = 200
    elif letter_index == 8:
        x = 600
        y = 200
    elif letter_index == 9:
        x = 800
        y = 200
    elif letter_index == 10:   #from this it is 2nd row
        x = 0
        y = 400
    elif letter_index == 11:
        x = 200
        y = 400
    elif letter_index == 12:
        x = 400
        y = 400
    elif letter_index == 13:
        x = 600
        y = 400
    elif letter_index == 14:
        x = 800
        y = 400
    
    
    width = 200
    height = 200
    thickness = 3
    
    
    #to highlight the letter which is selected with some different color.
    if letter_light is True:
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (255, 255, 255), -1)
    else:    
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (0,0, 255), thickness)
    
    #settings of the text
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_thickness = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_thickness)[0]  #gives size of the letter
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x   #pixels are in integer.
    text_y = int((height + height_text) / 2) + y
    cv2.putText(keyboard, text , (text_x, text_y), font_letter , font_scale, (0, 0, 255), font_thickness)


for i in range(15):   #as I am drawing 15 letters and we are lighting for letter 5 only.
    if i== 5:
        light  =True
    else:
        light = False
    letter(i, keys_set1[i], light)  


cv2.imshow('keyboard', keyboard)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Linking both the above codes - detection of gaze and virtual keyboard and adding a lighting feature which goes till the end of the keyboard and then starts again from the bginning. 

In [36]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily


cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#keyboard settings
keyboard = np.zeros((600, 1000, 3), np.uint8)   #creation of black screen
keys_set1 = {0 : "Q", 1: 'W', 2: 'E', 3: 'R', 4: 'T',
            5: 'A', 6: 'S', 7: 'D', 8: 'F', 9: 'G',
            10: 'Z', 11: 'X', 12: 'C', 13: 'V', 14: 'B'}


def letter(letter_index, text, letter_light):  #this function is used to draw bixes of rectangles and the text inside it. 
    #Keys
    if letter_index == 0:  #0th row
        x = 0
        y = 0
    elif letter_index == 1:
        x = 200
        y = 0
    elif letter_index == 2:
        x = 400
        y = 0
    elif letter_index == 3:
        x = 600
        y = 0  
    elif letter_index == 4:
        x = 800
        y = 0
    elif letter_index == 5:   #from this it is 1st row
        x = 0
        y = 200
    elif letter_index == 6:
        x = 200
        y = 200
    elif letter_index == 7:
        x = 400
        y = 200
    elif letter_index == 8:
        x = 600
        y = 200
    elif letter_index == 9:
        x = 800
        y = 200
    elif letter_index == 10:   #from this it is 2nd row
        x = 0
        y = 400
    elif letter_index == 11:
        x = 200
        y = 400
    elif letter_index == 12:
        x = 400
        y = 400
    elif letter_index == 13:
        x = 600
        y = 400
    elif letter_index == 14:
        x = 800
        y = 400
    
    
    width = 200
    height = 200
    thickness = 3
    
    
    #to highlight the letter which is selected with some different color.
    if letter_light is True:
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (255, 255, 255), -1)
    else:    
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (0,0, 255), thickness)
    
    #settings of the text
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_thickness = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_thickness)[0]  #gives size of the letter
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x   #pixels are in integer.
    text_y = int((height + height_text) / 2) + y
    cv2.putText(keyboard, text , (text_x, text_y), font_letter , font_scale, (0, 0, 255), font_thickness)



def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

def get_gaze_ratio(eye_points, landmarks):
    
        #Gaze detection - drawing polygon on the left eye and getting the exact region of the eye
        left_eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                                   (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                                   (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                                   (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                                   (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                                   (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
        #drawing polygon on the left eye
        #cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
        
        
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)  #this is a black screen of size same as our frame
        #We will create the eye region in this mask
        cv2.polylines(mask, [left_eye_region], True, (255,255,255), 2)  #creating the polygon of the eye region
        cv2.fillPoly(mask, [left_eye_region], 255)  #filling the polygon
        #mask gives exact eye, so we put this mask over the frame so as to get exact eye and the rest extra portion of eye is removed.
        eye = cv2.bitwise_and(gray, gray, mask = mask)
        
        
        #detecting the eye region
        min_x = np.min(left_eye_region[:, 0])  #left most pt is min x
        max_x = np.max(left_eye_region[:, 0])  #rightt most pt is max x
        min_y = np.min(left_eye_region[:, 1])  #top most pt is min y
        max_y = np.max(left_eye_region[:, 1])  #bottomt most pt is max y
        
        gray_eye = eye[min_y : max_y, min_x : max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
        
        #dividing the threshold into 2 parts - left and write threshold
        height, width = threshold_eye.shape
        left_side_threshold = threshold_eye[0: height, 0: int(width/2)]
        right_side_threshold = threshold_eye[0: height, int(width/2): width]
        
        #calculating the amount of white part in both the thresholds - left and right
        left_side_white = cv2.countNonZero(left_side_threshold)
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        #handling zero divison error
        if left_side_white == 0:
            gaze_ratio = 1   #means we are looking at the left
        elif right_side_white == 0:
            gaze_ratio = 5   #means we are looking at the right
        else:
            gaze_ratio = left_side_white/ right_side_white
        

        return gaze_ratio
    
    

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

#Counting frames
frames = 0
letter_index = 0

while True:
    
    _, frame = cap.read()
    frame = cv2.resize(frame, None, fx = 0.5, fy = 0.5)  #resizing the size to its half so as to remove computational loss
    keyboard[:] = (0, 0, 0)   #each time I fill the keyboard with black color so that the previous letter after lighting one letter becomes black.
    #showing direction when we turn the face eithre left and right
    frames += 1   #incrementing the frames count everytime whenever there is a new frame.
    new_frame = np.zeros((500, 500, 3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        #Detect blinking
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0))
        
        
        #gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio = (gaze_ratio_left_eye + gaze_ratio_right_eye)/ 2
        
        if gaze_ratio < 0.8: 
            cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (0, 0, 255)
        elif 0.8 < gaze_ratio < 1.2:
            cv2.putText(frame, "CENTER", (50, 100), font, 2, (0,0,255), 3)
        else:
            cv2.putText(frame, "LEFT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (255, 0, 0)
        
        
    #letters
    if frames == 10:
        letter_index += 1  #so it shows the different letter.
        frames = 0
        
    if letter_index == 15:
        letter_index = 0   #so that after the lighting all letters till the end it comes to the start index
        
        
    for i in range(15):   #as I am drawing 15 letters.
        if i == letter_index:   #whenever the index changes, then change the letter color. 
            light = True
        else:
            light = False
        letter(i, keys_set1[i], light) 
        
        
        
        
    cv2.imshow("Frame", frame)
    cv2.imshow('New Frame', new_frame)
    cv2.imshow('Virtual Keyboard', keyboard)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# To press the letter when we are blinking our eyes.

In [39]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily


cap = cv2.VideoCapture(0)
board = np.zeros((500, 500), np.uint8)  #I display the text on this board.
board[:] = 255  #making this board white

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#keyboard settings
keyboard = np.zeros((600, 1000, 3), np.uint8)   #creation of black screen
keys_set1 = {0 : "Q", 1: 'W', 2: 'E', 3: 'R', 4: 'T',
            5: 'A', 6: 'S', 7: 'D', 8: 'F', 9: 'G',
            10: 'Z', 11: 'X', 12: 'C', 13: 'V', 14: 'B'}


def letter(letter_index, text, letter_light):  #this function is used to draw bixes of rectangles and the text inside it. 
    #Keys
    if letter_index == 0:  #0th row
        x = 0
        y = 0
    elif letter_index == 1:
        x = 200
        y = 0
    elif letter_index == 2:
        x = 400
        y = 0
    elif letter_index == 3:
        x = 600
        y = 0  
    elif letter_index == 4:
        x = 800
        y = 0
    elif letter_index == 5:   #from this it is 1st row
        x = 0
        y = 200
    elif letter_index == 6:
        x = 200
        y = 200
    elif letter_index == 7:
        x = 400
        y = 200
    elif letter_index == 8:
        x = 600
        y = 200
    elif letter_index == 9:
        x = 800
        y = 200
    elif letter_index == 10:   #from this it is 2nd row
        x = 0
        y = 400
    elif letter_index == 11:
        x = 200
        y = 400
    elif letter_index == 12:
        x = 400
        y = 400
    elif letter_index == 13:
        x = 600
        y = 400
    elif letter_index == 14:
        x = 800
        y = 400
    
    
    width = 200
    height = 200
    thickness = 3
    
    
    #to highlight the letter which is selected with some different color.
    if letter_light is True:
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (255, 255, 255), -1)
    else:    
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (0,0, 255), thickness)
    
    #settings of the text
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_thickness = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_thickness)[0]  #gives size of the letter
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x   #pixels are in integer.
    text_y = int((height + height_text) / 2) + y
    cv2.putText(keyboard, text , (text_x, text_y), font_letter , font_scale, (0, 0, 255), font_thickness)



def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

def get_gaze_ratio(eye_points, landmarks):
    
        #Gaze detection - drawing polygon on the left eye and getting the exact region of the eye
        left_eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                                   (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                                   (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                                   (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                                   (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                                   (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
        #drawing polygon on the left eye
        #cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
        
        
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)  #this is a black screen of size same as our frame
        #We will create the eye region in this mask
        cv2.polylines(mask, [left_eye_region], True, (255,255,255), 2)  #creating the polygon of the eye region
        cv2.fillPoly(mask, [left_eye_region], 255)  #filling the polygon
        #mask gives exact eye, so we put this mask over the frame so as to get exact eye and the rest extra portion of eye is removed.
        eye = cv2.bitwise_and(gray, gray, mask = mask)
        
        
        #detecting the eye region
        min_x = np.min(left_eye_region[:, 0])  #left most pt is min x
        max_x = np.max(left_eye_region[:, 0])  #rightt most pt is max x
        min_y = np.min(left_eye_region[:, 1])  #top most pt is min y
        max_y = np.max(left_eye_region[:, 1])  #bottomt most pt is max y
        
        gray_eye = eye[min_y : max_y, min_x : max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
        
        #dividing the threshold into 2 parts - left and write threshold
        height, width = threshold_eye.shape
        left_side_threshold = threshold_eye[0: height, 0: int(width/2)]
        right_side_threshold = threshold_eye[0: height, int(width/2): width]
        
        #calculating the amount of white part in both the thresholds - left and right
        left_side_white = cv2.countNonZero(left_side_threshold)
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        #handling zero divison error
        if left_side_white == 0:
            gaze_ratio = 1   #means we are looking at the left
        elif right_side_white == 0:
            gaze_ratio = 5   #means we are looking at the right
        else:
            gaze_ratio = left_side_white/ right_side_white
        

        return gaze_ratio
    
    

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

#Counting frames
frames = 0
letter_index = 0
blinking_frames = 0
text = ""    #this is the text which is displayed each time I close my eye

while True:
    
    _, frame = cap.read()
    frame = cv2.resize(frame, None, fx = 0.5, fy = 0.5)  #resizing the size to its half so as to remove computational loss
    keyboard[:] = (0, 0, 0)   #each time I fill the keyboard with black color so that the previous letter after lighting one letter becomes black.
    #showing direction when we turn the face eithre left and right
    frames += 1   #incrementing the frames count everytime whenever there is a new frame.
    new_frame = np.zeros((500, 500, 3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    active_letter = keys_set1[letter_index]    #this is the letter when I bink my eyes
            
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        #Detect blinking
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:  #If I blink the eye for longet time then only the letter is pressed.
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0), thickness = 3)
            blinking_frames += 1  #the number of frames till we are blinking
            frames -= 1  #when we are blinking then the frame detection should stop, so the letter wont change.
            
            if blinking_frames == 5:  #if I blink for 5 frames the add that letter to the text
                text += active_letter
        
        else:
            blinking_frames = 0  #if I am not blinking then it is = 0.
            
        
        #gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio = (gaze_ratio_left_eye + gaze_ratio_right_eye)/ 2
        
        
        """
        if gaze_ratio < 0.8: 
            cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (0, 0, 255)
        elif 0.8 < gaze_ratio < 1.2:
            cv2.putText(frame, "CENTER", (50, 100), font, 2, (0,0,255), 3)
        else:
            cv2.putText(frame, "LEFT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (255, 0, 0)  
        """
        
        
    #letters
    if frames == 15:
        letter_index += 1  #so it shows the different letter.
        frames = 0
        
    if letter_index == 15:
        letter_index = 0   #so that after the lighting all letters till the end it comes to the start index
        
        
    for i in range(15):   #as I am drawing 15 letters.
        if i == letter_index:   #whenever the index changes, then change the letter color. 
            light = True
        else:
            light = False
        letter(i, keys_set1[i], light) 
        
    cv2.putText(board, text, (10, 100), font, 4, 0, 3)
        
        
        
        
    cv2.imshow("Frame", frame)
    #cv2.imshow('New Frame', new_frame)
    cv2.imshow('Virtual Keyboard', keyboard)
    cv2.imshow('Board', board)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# In above there is a problem that the user does not get to know when the letter is pressed as his/ her eyes are closed.
So I added a sound in this through which I can know when the letter is pressed.

In [ ]:
#Adding audios - sample code

import time
import pyglet

sound = pyglet.media.load('Sound.wav')
sound.play()
pyglet.app.run()
#time.sleep(1)


In [ ]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily
import pyglet  #for playing audios
import time

#loading the sounds
sound = pyglet.media.load('sound.wav', streaming = False)  #streaming = False is done to use this sound multiple times while streaming.
left_sound = pyglet.media.load('left.wav', streaming = False)
right_sound = pyglet.media.load('right.wav', streaming = False)

cap = cv2.VideoCapture(0)
board = np.zeros((500, 500), np.uint8)  #I display the text on this board.
board[:] = 255  #making this board white

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

#keyboard settings
keyboard = np.zeros((600, 1000, 3), np.uint8)   #creation of black screen
keys_set1 = {0 : "Q", 1: 'W', 2: 'E', 3: 'R', 4: 'T',
            5: 'A', 6: 'S', 7: 'D', 8: 'F', 9: 'G',
            10: 'Z', 11: 'X', 12: 'C', 13: 'V', 14: 'B'}


def letter(letter_index, text, letter_light):  #this function is used to draw bixes of rectangles and the text inside it. 
    #Keys
    if letter_index == 0:  #0th row
        x = 0
        y = 0
    elif letter_index == 1:
        x = 200
        y = 0
    elif letter_index == 2:
        x = 400
        y = 0
    elif letter_index == 3:
        x = 600
        y = 0  
    elif letter_index == 4:
        x = 800
        y = 0
    elif letter_index == 5:   #from this it is 1st row
        x = 0
        y = 200
    elif letter_index == 6:
        x = 200
        y = 200
    elif letter_index == 7:
        x = 400
        y = 200
    elif letter_index == 8:
        x = 600
        y = 200
    elif letter_index == 9:
        x = 800
        y = 200
    elif letter_index == 10:   #from this it is 2nd row
        x = 0
        y = 400
    elif letter_index == 11:
        x = 200
        y = 400
    elif letter_index == 12:
        x = 400
        y = 400
    elif letter_index == 13:
        x = 600
        y = 400
    elif letter_index == 14:
        x = 800
        y = 400
    
    
    width = 200
    height = 200
    thickness = 3
    
    
    #to highlight the letter which is selected with some different color.
    if letter_light is True:
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (255, 255, 255), -1)
    else:    
        cv2.rectangle(keyboard, (x + thickness, y + thickness), (x + width - thickness, y + height - thickness), (0,0, 255), thickness)
    
    #settings of the text
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_thickness = 4
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_thickness)[0]  #gives size of the letter
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x   #pixels are in integer.
    text_y = int((height + height_text) / 2) + y
    cv2.putText(keyboard, text , (text_x, text_y), font_letter , font_scale, (0, 0, 255), font_thickness)



def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

def get_gaze_ratio(eye_points, landmarks):
    
        #Gaze detection - drawing polygon on the left eye and getting the exact region of the eye
        left_eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                                   (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                                   (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                                   (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                                   (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                                   (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
        #drawing polygon on the left eye
        #cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
        
        
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)  #this is a black screen of size same as our frame
        #We will create the eye region in this mask
        cv2.polylines(mask, [left_eye_region], True, (255,255,255), 2)  #creating the polygon of the eye region
        cv2.fillPoly(mask, [left_eye_region], 255)  #filling the polygon
        #mask gives exact eye, so we put this mask over the frame so as to get exact eye and the rest extra portion of eye is removed.
        eye = cv2.bitwise_and(gray, gray, mask = mask)
        
        
        #detecting the eye region
        min_x = np.min(left_eye_region[:, 0])  #left most pt is min x
        max_x = np.max(left_eye_region[:, 0])  #rightt most pt is max x
        min_y = np.min(left_eye_region[:, 1])  #top most pt is min y
        max_y = np.max(left_eye_region[:, 1])  #bottomt most pt is max y
        
        gray_eye = eye[min_y : max_y, min_x : max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
        
        #dividing the threshold into 2 parts - left and write threshold
        height, width = threshold_eye.shape
        left_side_threshold = threshold_eye[0: height, 0: int(width/2)]
        right_side_threshold = threshold_eye[0: height, int(width/2): width]
        
        #calculating the amount of white part in both the thresholds - left and right
        left_side_white = cv2.countNonZero(left_side_threshold)
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        #handling zero divison error
        if left_side_white == 0:
            gaze_ratio = 1   #means we are looking at the left
        elif right_side_white == 0:
            gaze_ratio = 5   #means we are looking at the right
        else:
            gaze_ratio = left_side_white/ right_side_white
        

        return gaze_ratio
    
    

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

#Counting frames
frames = 0
letter_index = 0
blinking_frames = 0
text = ""    #this is the text which is displayed each time I close my eye
keyboard_selected = "left"    #This is done so as to jst play the sound once when we change direction
last_keyboard_selected = "left"   #To know the direction of last time

while True:
    
    _, frame = cap.read()
    frame = cv2.resize(frame, None, fx = 0.5, fy = 0.5)  #resizing the size to its half so as to remove computational loss
    keyboard[:] = (0, 0, 0)   #each time I fill the keyboard with black color so that the previous letter after lighting one letter becomes black.
    #showing direction when we turn the face eithre left and right
    frames += 1   #incrementing the frames count everytime whenever there is a new frame.
    new_frame = np.zeros((500, 500, 3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    active_letter = keys_set1[letter_index]    #this is the letter when I bink my eyes
            
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        #Detect blinking
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:  #If I blink the eye for longet time then only the letter is pressed.
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0), thickness = 3)
            blinking_frames += 1  #the number of frames till we are blinking
            frames -= 1  #when we are blinking then the frame detection should stop, so the letter wont change.
            
            #Typing letter
            if blinking_frames == 5:  #if I blink for 5 frames the add that letter to the text
                text += active_letter
                sound.play()
                pyglet.app.run()
        
        else:
            blinking_frames = 0  #if I am not blinking then it is = 0.
            
        
        #gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio = (gaze_ratio_left_eye + gaze_ratio_right_eye)/ 2
        
        
        
        if gaze_ratio < 0.8:
            keyboard_selected = "right"
            if keyboard_selected != last_keyboard_selected:
                right_sound.play()
                pyglet.app.run()
                last_keyboard_selected = keyboard_selected
                
        #elif 0.8 < gaze_ratio < 1.2:
            #cv2.putText(frame, "CENTER", (50, 100), font, 2, (0,0,255), 3)
        else:
            keyboard_selected = "left"
            if keyboard_selected != last_keyboard_selected:
                left_sound.play() 
                pyglet.app.run()
                last_keyboard_selected = keyboard_selected
       
        
        
    #letters
    if frames == 15:
        letter_index += 1  #so it shows the different letter.
        frames = 0
        
    if letter_index == 15:
        letter_index = 0   #so that after the lighting all letters till the end it comes to the start index
        
        
    for i in range(15):   #as I am drawing 15 letters.
        if i == letter_index:   #whenever the index changes, then change the letter color. 
            light = True
        else:
            light = False
        letter(i, keys_set1[i], light) 
        
    cv2.putText(board, text, (10, 100), font, 4, 0, 3)
        
        
        
        
    cv2.imshow("Frame", frame)
    cv2.imshow('Virtual Keyboard', keyboard)
    cv2.imshow('Board', board)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()

# Creating both the halves of  the keyboard. When I look at the left then I have the left part of the keyboard and if I look at the right then Right part of the keyboarad appears and to get back to the menu screen I blink my eyes on the "<" symbol and to press space I use "_" symbol.

The above code is in final code file